# Interactive Data Visualization by Example

## Table of Contents

* [Introduction](#Introduction)
* [Prologue: Examining the Data](#Prologue:-Examining-The-Data)
* [Plot 1: Average Bechdel Score over Time](#Plot-1:-Average-Bechdel-Score-over-Time)
    * [Recreating Plot 1](#Recreating-Plot-1) (and intro to basic Bokeh plots)
    * [Manipulating Plot 1: Or, The Treachery of Ordinal Data ](#Manipulating-Plot-1:-Or,-The-Treachery-of-Ordinal-Data)
    * [Improving Plot 1: Rolling Averages](#Improving-Plot-1:-Rolling-Averages)
    * [Improving Plot 1: Showing Variance](#Improving-Plot-1:-Showing-Variance)
    * [Making Plot 1 Interactive](#Making-Plot-1-Interactive) (and intro to Bokeh plot tools)

## Introduction

On April 1, 2014, Walt Hickey wrote an article titled [*The Dollar-And-Cents Case Against Hollywood’s Exclusion of Women*](https://fivethirtyeight.com/features/the-dollar-and-cents-case-against-hollywoods-exclusion-of-women/), which looked at how Hollywood blockbusters have faired in the [Bechdel Test](https://en.wikipedia.org/wiki/Bechdel_test). The test asks a simple question: Does a film

1. Contain two named female charaters
2. Who talk to each other
3. About something other than a male character?

Applying this criterion to films over the last fifty years, FiveThirtyEight concludes that films that pass the test are not only better representations of gender equality, but also tend to have a higher return on investment (ROI). Because of this, it is in Hollywood's interest to produce films with a more diverse female cast.

Several weeks later, on April 21, 2014, Brian Keegan wrote a response titled [*The Need for Openness in Data Journalism*](https://nbviewer.jupyter.org/github/brianckeegan/Bechdel/blob/master/Bechdel_test.ipynb). Using the original article as a case study, Brian point as much about female representation in film (on which he mostly validated Walk Hickey's conclusions, with some caveats) as it is on the importance for making the source code and data for this kind of analysis available. Five Thirty Eight themselves took notice of the response and [linked to it from their site](https://fivethirtyeight.com/features/the-bechdel-test-checking-our-work/), and later made their articles available for replication on their [Github page](https://github.com/fivethirtyeight/data/).

This document follows a similar vein; I will use the Bechdel Test data as a worked example of how to create visually appealing plots, and also as a tutorial for creating *interactive* data visualizations. I was spurred to write this document because of a Communication Computational Concepts course I taught at Occidental College in Spring 2018, when I couldn't find good introductions for basic data literacy. My goal here is to not only demonstrate what a good visualization might look like, but also to explain the *thought process* in creating it. Finally, a side effect of writing this document is to illustrate how I use/organize a Jupyter notebook.

The overall structure of this document is to take the four main plots in Brian Keegan's document, and for each one:

1. recreate the plot with Bokeh
2. improve the plot by making it more readable and/or information dense
3. create an interactive version that would allow reader exploration

Although this document can be viewed statically with the [Jupyter Notebook Viewer](http://nbviewer.jupyter.org/), the interaction visualizations requires a running Jupyter notebook with Python 3. Those interested can check out the [Github repository](https://github.com/justinnhli/blog-code/tree/master/2018/05/bechdel) to run it locally.

Note: I am assuming that the reader are passingly fluent with pandas DataFrames (and Python, of course). DataFrame operations will mostly not be explained, but feel free to look up the functions in the [DataFrame API reference](https://pandas.pydata.org/pandas-docs/stable/api.html) as we go along. If you need a refresher on pandas, I recommend working through [First Python Notebook](http://www.firstpythonnotebook.org/).

## Prologue: Examining The Data

In [ ]:
import math

import numpy as np
import pandas as pd
from IPython.core.display import HTML
from bokeh.plotting import figure, ColumnDataSource, show, output_notebook
from bokeh.palettes import all_palettes as palettes
from bokeh.layouts import gridplot

output_notebook()

Before we get dive into making pretty graphs, we need to understand the data that we are using to create those graphs. The data I am using here are the same ones that Brian Keegan collected in April 2014. Specifically, he used data from four different sources:

* [BechdelTest.com](http://bechdeltest.com/) for how films scored on the Bechdel test
* [The-Numbers.com](https://www.the-numbers.com/) for the budget and revenue data
* [The Bureau of Labor Statistics](https://www.bls.gov/) for historical inflation data
* [The Open Movie Database (OMBD)](https://www.omdbapi.com/) for rating data

The following code is copied almost verbatim from Brian Keegan.

Two pedagogical notes about this code:

1. I tend to write one-off functions that encapsulate most of the processing, then call that function and save the results into a variable. This keeps the main namespace relatively clean.

2. I have found that it is a good idea to read the data in one cell, then manipulate copies of it in subsequent cells. This allows for quicker iteration of code, since you don't have to re-read the data if you screw up. This also prevents the `a value is trying to be set on a copy of a slice from a DataFrame` error. What I tend to do is write the code outside a function and test it as I go, then wrap it in a function when I get it working.

In [ ]:
def read_data():

    def read_revenues():
        return pd.read_csv('data/revenue.csv', encoding='utf8', index_col=0)

    def read_inflations():
        inflation_df = pd.read_csv('data/cpi.csv', index_col='Year')
        inflation_df = inflation_df['Annual']
        return dict(inflation_df.loc[2014] / inflation_df)

    def read_bechdel():
        bechdel_df = pd.read_json('data/bechdel.json')
        bechdel_df['imdbid'] = bechdel_df['imdbid'].dropna().apply(int)
        bechdel_df = bechdel_df.set_index('imdbid')
        bechdel_df.dropna(subset=['title'], inplace=True)
        bechdel_df = bechdel_df[['rating', 'title']]
        return bechdel_df

    def read_imdb():

        def runtime_to_minutes(runtime):
            if 'h' in runtime:
                hours, minutes = runtime.split('h')
                return str(int(hours) * 60 + int(minutes))
            else:
                return runtime

        # Read the data in
        imdb_df = pd.read_json('data/imdb_data.json')

        # Drop non-movies
        imdb_df = imdb_df[imdb_df['Type'] == 'movie']

        # Drop movies with unknown release dates
        imdb_df = imdb_df[imdb_df['Released'] != 'N/A']

        # Convert to datetime objects
        imdb_df['Released'] = pd.to_datetime(imdb_df['Released'], format="%d %b %Y")

        # Drop errant identifying characters in the ID field
        imdb_df['imdbID'] = imdb_df['imdbID'].str.slice(start=2)

        # Remove the " min" at the end of Runtime entries so we can convert to ints
        imdb_df['Runtime'] = imdb_df['Runtime'].str.slice(stop=-4).replace('', np.nan)

        # Convert errant runtimes to minutes
        imdb_df['Runtime'] = imdb_df['Runtime'].astype(str).apply(runtime_to_minutes)

        # Blank out non-MPAA or minor ratings (NC-17, X)
        imdb_df['Rated'] = imdb_df['Rated'].replace(
            to_replace=[
                'N/A',
                'Not Rated',
                'Approved',
                'Unrated',
                'TV-PG',
                'TV-G',
                'TV-14',
                'TV-MA',
                'NC-17',
                'X',
            ],
            value=np.nan,
        )

        # Convert Release datetime into new columns for year, month, and week
        imdb_df['Year'] = imdb_df['Released'].apply(lambda date: date.year)
        imdb_df['Month'] = imdb_df['Released'].apply(lambda date: date.month)
        imdb_df['Week'] = imdb_df['Released'].apply(lambda date: date.week)

        # Convert the series to float
        imdb_df['Runtime'] = imdb_df['Runtime'].apply(float)

        # Convert the imdbVotes strings into float
        imdb_df['imdbVotes'] = imdb_df['imdbVotes'].replace('N/A', np.nan).dropna().apply(
            lambda s: float(s.replace(',', ''))
        )

        # Take the Metascore formatted as string containing "N/A", convert to float
        # Also divide by 10 to make effect sizes more comparable
        imdb_df['Metascore'] = imdb_df['Metascore'].dropna().replace('N/A', np.nan).dropna().apply(float) / 10

        # Take the imdbRating formatted as string containing "N/A", convert to float
        imdb_df['imdbRating'] = imdb_df['imdbRating'].dropna().replace('N/A', np.nan).dropna().apply(float)

        # Create a dummy variable for English language
        imdb_df['English'] = (imdb_df['Language'] == u'English').astype(int)
        imdb_df['USA'] = (imdb_df['Country'] == u'USA').astype(int)

        # Convert imdb_ID to int, set it as the index
        imdb_df['imdbID'] = imdb_df['imdbID'].dropna().apply(int)
        imdb_df = imdb_df.set_index('imdbID')

        return imdb_df

    def get_combined_data():
        revenue_df = read_revenues()
        inflation_dict = read_inflations()
        bechdel_df = read_bechdel()
        imdb_df = read_imdb()
        df = imdb_df.join(bechdel_df, how='inner').reset_index()
        df = pd.merge(df, revenue_df, left_on=['Title', 'Year'], right_on=['Movie', 'Year'])
        df['Year'] = df['Released_x'].apply(lambda date: date.year)
        df['Adj_Revenue'] = df.apply(lambda row: row['Revenue'] * inflation_dict[row['Year']], axis=1)
        df['Adj_Budget'] = df.apply(lambda row: row['Budget'] * inflation_dict[row['Year']], axis=1)
        return df

    return get_combined_data()


raw_df = read_data()
raw_df.tail(2).T

A commonly-overlooked part of creating data visualizations is *understanding the data*. There are at least two things worth noting here:

First, since I am using Brian's original 2014 data, all dollar amounts will be in 2014 dollars.

Second, and more importantly, it's worth understanding what Brian called the "Bechdel score" or the "Bechdel dimension". This is the `rating` column (row in the above sample) is the rating from bechdeltest.com. As Brian quoted from their API: 

> The actual score. Number from 0 to 3.
> * 0.0 means no two women,
> * 1.0 means no talking, 
> * 2.0 means talking about a man, 
> * 3.0 means it passes the test.

Note that *the data is **ordinal** and not numeric*. The only semantic information available is that a film with no women is "worse" than a film with no women talking, which is in turn worse than a film with women talking about a man. The mapping to 0, 1, 2, and 3 is *arbitrary* - we could just as well map it to -7, 0.5, 0.6, 1000; all that matters is the numbers are increasing. As [Wikipedia page on ordinal data](https://en.wikipedia.org/wiki/Ordinal_data#General) says, "the use of means and standard deviations ... [is] not appropriate".

This leads directly to Brian's first data plot...

## Plot 1: Average Bechdel Score over Time

Brian has a plot that compares the data he collected with that of FiveThirtyEight's. Although we can now get the original data from FiveThirtyEight's repository, we will ignore that plot for the sake of sticking with the information Brian had at the time of his writing.

The next plot that Brian presents is the average Bechdel score over time:

![Original Average Bechdel Test over Time Plot](original-plots/1a.png)

Remember that ordinal data stuff from the last section, and how we shouldn't use means and standard deviations? *This plot violates that rule.* As we will see, the average Bechdel score (and so the plot itself) is almost meaningless, and its appearance can be changed drastically with a very different mapping of Bechdel categories to numbers.

But first, let's recreate this plot in Bokeh.

### Recreating Plot 1

The following function takes the raw DataFrame from before and extracts only the year and rating information:

In [ ]:
def prepare_plot_1_data():
    
    # create a DataFrame that will contiain the data to be plotted
    plot_df = raw_df.copy()

    # select only the columns we will need
    plot_df = plot_df[['Year', 'rating']]

    # find the mean rating for each year
    plot_df = plot_df.groupby(by='Year').mean()

    # reset the index so Bokeh can understand the data
    plot_df = plot_df.reset_index()
    
    return plot_df

prepare_plot_1_data().head()

To plot the data, we will use the [Bokeh](https://bokeh.pydata.org/) library. Although there are a lot of plotting libraries to choose from, including [Matplotlib](https://matplotlib.org/) (which is the library that pandas uses for their [plotting functions](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.plot.html)), [Seaborn](https://seaborn.pydata.org/), [Plotly](https://plot.ly/), and more recently [ggplot](http://ggplot.yhathq.com/) and [Altair](https://altair-viz.github.io/), I personally like Bokeh's approach of layering glyphs on top of each other, which makes it easy to add additional visual components. For the same reason, I never liked the old [high-level charts API](https://bokeh.github.io/blog/2017/6/13/release-0-12-6/) even before it was deprecated.

Creating a Bokeh plot involves three steps (after wrangling the data into shape):

1. Creating the Figure object. This is simply a call to the [`figure()`](https://bokeh.pydata.org/en/latest/docs/reference/plotting.html#bokeh.plotting.figure.figure) factory function. Some plot-level attributes such as size, title, axes, etc. can be set at this point.

2. Creating glyphs that represent the data. This involves calling at least one of the many glyph methods [in the Figure class](https://bokeh.pydata.org/en/latest/docs/reference/plotting.html#bokeh.plotting.figure.Figure). Each glyph takes different arguments; a [`circle()` glyph](https://bokeh.pydata.org/en/latest/docs/reference/plotting.html#bokeh.plotting.figure.Figure.circle), for example, takes `x` and `y`, while a [`vbar()` glyph](https://bokeh.pydata.org/en/latest/docs/reference/plotting.html#bokeh.plotting.figure.Figure.vbar) (a vertical rectangle) takes `x`, `width`, `top`, and `bottom`. Central to this API is that each glyph takes a `source` keyword argument, which tells the glyph where to get the data from. Most of the time, if you are using using Bokeh with pandas, this source will be a [ColumnDataSource](https://bokeh.pydata.org/en/latest/docs/reference/models/sources.html#bokeh.models.sources.ColumnDataSource) created from a pandas DataFrame.

3. Showing the figure. In a Jupyter notebook, this means two things. First, to tell Bokeh that you are working in a Jupyter notebook, the [`output_notebook()`](https://bokeh.pydata.org/en/latest/docs/reference/io.html#bokeh.io.output_notebook) function must be called. I tend to do this at the start right after I import the necessary libraries. Once we have that, we call the [`show()`](https://bokeh.pydata.org/en/latest/docs/reference/io.html#bokeh.io.show) function for each figure we want to display.

We follow these three steps in the following function:

In [ ]:
def create_plot_1():
    
    # get the data for the plot
    plot_df = prepare_plot_1_data()
    
    # convert it to a ColumnDataSource
    data_source = ColumnDataSource(plot_df)
    
    # create the figure (step 1)
    fig = figure()
    
    # create the glyph, in this case, a line (step 2)
    fig.line(
        # use the 'Year' column as the x value
        x='Year',
        # use the 'rating' column as the y value
        y='rating',
        # use the converted ColumnDataSource as the source of columns
        source=data_source,
    )
    
    return fig

# show the plot (step 3)
show(create_plot_1())

One nice thing about Bokeh is that the plots are semi-interactive by default - you can drag the plot to pan around, use the tools on the right to zoom in, or reset the plot to the original view.

We used the Bokeh defaults for this first attempt, but we can style it a bit more. In the code below, we will change the plot size, set the domain and range, and add axis labels.

In [ ]:
def create_plot_1():
    
    # get the data for the plot
    plot_df = prepare_plot_1_data()
    
    # convert it to a ColumnDataSource
    data_source = ColumnDataSource(plot_df)
    
    # create the figure (step 1)
    fig = figure(
        # set the size of the plot
        width=600, height=400,
        # set the domain and range
        x_range=[1910, 2020],
        y_range=[0, 3],
        # set axis labels
        x_axis_label='Year',
        y_axis_label='Avg. Bechdel Test',
    )
    
    # create the glyph, in this case, a line (step 2)
    fig.line(
        # use the 'Year' column as the x value
        x='Year',
        # use the 'rating' column as the y value
        y='rating',
        # use the converted ColumnDataSource as the source of columns
        source=data_source,
    )
    
    return fig

# show the plot (step 3)
show(create_plot_1())

Looking back at the original plot, we can see that we've recreated most of the visual elements:

![Original Average Bechdel Test over Time Plot](original-plots/1a.png)

The one element we *didn't* get is the line width - if you look at the original closely, you will see that the lines gets thicker towards the right. Brian never explains what this means in his text, but if you look at his code, you will see this in cell 37:

    lines = LineCollection(lines, linewidths=dict(num_movies).values())

It seems like the line width represents how many movies were averaged over. Unfortunately, Bokeh does not offer an easy way to varying the line width. What we have to do instead is draw each line separately by looping over the rows:

In [ ]:
def plot_1():
    
    # prepare the data
    plot_df = raw_df.copy()
    plot_df = plot_df[['Year', 'rating']]
    # calculate the mean and number of ratings separately then combine them
    plot_df = pd.concat(
        [
            plot_df.groupby(by='Year').mean()['rating'],
            plot_df.groupby(by='Year').count()['rating'],
        ],
        axis=1,
    )
    plot_df = plot_df.reset_index()
    # rename the columns appropriately
    plot_df.columns = ['Year', 'rating', 'count']
    
    fig = figure(
        width=600, height=400,
        x_range=[1910, 2020],
        y_range=[0, 3],
        x_axis_label='Year',
        y_axis_label='Avg. Bechdel Test',
    )
    
    # convert the DataFrame into a list of rows
    # take each pair of rows and plot them as a line
    rows_list = list(plot_df.itertuples())
    for start, end in zip(rows_list[:-1], rows_list[1:]):
        fig.line(
            x=[start[1], end[1]],
            y=[start[2], end[2]],
            line_width=start[3],
        )
        
        
    return fig

show(plot_1())

This is almost there, except that the line on the right side are too thick. On the other hand, just dividing the width by a constant makes the other lines too thin. After some experimentation, it turns out using the log of the count gives the appropriate appearance:

In [ ]:
def plot_1():
    
    # prepare the data
    plot_df = raw_df.copy()
    plot_df = plot_df[['Year', 'rating']]
    # calculate the mean and number of ratings separately then combine them
    plot_df = pd.concat(
        [
            plot_df.groupby(by='Year').mean()['rating'],
            plot_df.groupby(by='Year').count()['rating'],
        ],
        axis=1,
    )
    plot_df = plot_df.reset_index()
    # rename the columns appropriately
    plot_df.columns = ['Year', 'rating', 'count']
    
    fig = figure(
        width=600, height=400,
        x_range=[1910, 2020],
        y_range=[0, 3],
        x_axis_label='Year',
        y_axis_label='Avg. Bechdel Test',
    )
    
    rows_list = list(plot_df.itertuples())
    for start, end in zip(rows_list[:-1], rows_list[1:]):
        fig.line(
            x=[start[1], end[1]],
            y=[start[2], end[2]],
            line_width=math.log(start[3]),
        )
        
        
    return fig

show(plot_1())

And there we have it: the original Bechdel test over time plot recreated in Bokeh.

### Manipulating Plot 1: Or, The Treachery of Ordinal Data 

Let's take a step back and go back to that point about ordinal data. I suggested that the plot we just recreated depends on how we convert the Bechdel test results to numbers. Specifically, the [bechdeltest.com API](https://bechdeltest.com/api/v1/doc) uses the following scoring system:

> * 0.0 means no two women,
> * 1.0 means no talking, 
> * 2.0 means talking about a man, 
> * 3.0 means it passes the test.

This might look acceptable at first glance, but what are the consequences of this scoring system? Consider, for example, what it means for the average Bechdel score to be 2.0, as it was in 2010. If there were a total of 3 films that year, it could mean that all three films had at least two women talking, but only about a man. Or, alternately, it could mean that 2 films pass the Bechdel test while 1 film did not even have two women ($\frac{2 * 3.0 + 1 * 0.0}{3} = \frac{6.0}{3} = 2.0$). For that matter, what does it even mean to add up the scores to take the average? Is three films where two women don't talk the same as one film that passes the Bechdel test ($3 * 1.0 = 1 * 3.0$)?

Perhaps a better example of the treachery of ordinal data is a peer evaluation survey that I use in my classes. I ask my students to rate each of their group members on the following scale:

> * No Show: No participation at all.
> * Superficial: Practically no participation.
> * Unsatisfactory: Consistently failed to show up or complete assignments, unprepared.
> * Deficient: Often failed to show up or complete assignments, rarely prepared.
> * Marginal: Sometimes failed to show up or complete assignments, rarely prepared.
> * Ordinary: Often did what they was supposed to do, minimally prepared and cooperative.
> * Satisfactory: Usually did what they was supposed to do, acceptably prepared and cooperative.
> * Very Good: Consistently did what they was supposed to do, very well prepared and cooperative.
> * Excellent: Consistently carried more than their fair share of the workload.

It might seem reasonable to simply take the perfect grade (100%) and the lowest possible grade (0%) and divide them equally between the ratings. Since there are 9 ratings, each rating will be 12.5% better than the last one:

<table>
    <tr>  <th>Rating</th>          <th>Percentage</th>  <th>Grade</th>  </tr>
    <tr>  <td>No Show</td>         <td>0.0%</td>        <td>F</td>      </tr>
    <tr>  <td>Superficial</td>     <td>12.5%</td>       <td>F</td>      </tr>
    <tr>  <td>Unsatisfactory</td>  <td>25.0%</td>       <td>F</td>      </tr>
    <tr>  <td>Deficient</td>       <td>37.5%</td>       <td>F</td>      </tr>
    <tr>  <td>Marginal</td>        <td>50.0%</td>       <td>F</td>      </tr>
    <tr>  <td>Ordinary</td>        <td>62.5%</td>       <td>D</td>      </tr>
    <tr>  <td>Satisfactory</td>    <td>75.0%</td>       <td>C</td>      </tr>
    <tr>  <td>Very Good</td>       <td>87.5%</td>       <td>B+</td>      </tr>
    <tr>  <td>Excellent</td>       <td>100.0%</td>      <td>A</td>      </tr>
</table>

On closer inspection, this doesn't seem right - a "satisfactory" is still only a solid C, and jump to the next rating ("Very Good") for a B+ is huge. If we go by intuition instead, we might come up with a mapping from rating to grade like this:

<table>
    <tr>  <th>Rating</th>          <th>Percentage</th>    <th>Grade</th>  </tr>
    <tr>  <td>No Show</td>         <td>0.0%</td>          <td>F</td>      </tr>
    <tr>  <td>Superficial</td>     <td>28.7%</td>         <td>F</td>      </tr>
    <tr>  <td>Unsatisfactory</td>  <td>43.5%</td>         <td>F</td>      </tr>
    <tr>  <td>Deficient</td>       <td>55.5%</td>         <td>F</td>      </tr>
    <tr>  <td>Marginal</td>        <td>66.0%</td>         <td>D</td>      </tr>
    <tr>  <td>Ordinary</td>        <td>75.4%</td>         <td>C</td>      </tr>
    <tr>  <td>Satisfactory</td>    <td>84.1%</td>         <td>B</td>      </tr>
    <tr>  <td>Very Good</td>       <td>92.3%</td>         <td>A-</td>     </tr>
    <tr>  <td>Excellent</td>       <td>100.0%</td>        <td>A</td>      </tr>
</table>

The conversion from rating to percentage is not the simple linear relationship, but an expoential one:

$$percentage = \left( \frac{rating}{8} \right) ^ {0.6}$$

where $rating$ is the index of the rating ("No Show" is 0 while "Excellent" is 8). In fact, we can vary the exponent to get an infinite number of mappings from ratings to percentages:

In [ ]:
def show_peer_eval_table():
    RATINGS = [        
        'No Show ',
        'Superficial ',
        'Unsatisfactory ',
        'Deficient ',
        'Marginal ',
        'Ordinary ',
        'Satisfactory ',
        'Very Good ',
        'Excellent ',
    ]
    exponents = [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]
    html = '<table style="margin:auto;">'
    html += '<tr><th>Rating</th>' + ''.join('<th>' + str(exponent) + '</th>' for exponent in exponents) + '</tr>'
    for index, rating in enumerate(RATINGS):
        html += '<tr>'
        html += '<td>' + rating + '</td>'
        for exponent in exponents:
            html += '<td>{:.1%}</td>'.format( (index / 8) ** exponent )
        html += '</tr>'
    html += '</table>'
    display(HTML(html))
        
show_peer_eval_table()

Of course, there's no reason to choose an exponential equation - we can just as easily use any polynomial or logarithm, or any other equation we can think of. The point here is that there are infinitely many ways to convert a [Likert scale](https://en.wikipedia.org/wiki/Likert_scale) to a number, and there may not be an obvious reason why we should pick one over the other.

Translating this back to the Bechdel Test, what if we used different numbers to convert the test results into numbers? What if instead of 0, 1, 2, 3, we used 0, 0, 0, 3? That is, what we if we only look at the binary of whether it passed the Bechdel test or not?

We can plot the averages from this new scale in red (we remove the line width for simplicity). This is also a good demonstration of how Bokeh allows us to layer one glyph on top of another.

In [ ]:
def plot_1_alternates(scale):
    
    # factor out a function to prepare the data
    def prepare_data_with_scale(scale):
        plot_df = raw_df.copy()
        plot_df = plot_df[['Year', 'rating']]
        # apply scale
        plot_df['rating'] = plot_df['rating'].apply(lambda i: scale[int(i)])
        plot_df = plot_df.groupby(by='Year').mean()
        plot_df = plot_df.reset_index()
        return plot_df
    
    fig = figure(
        width=600, height=400,
        x_range=[1910, 2020],
        y_range=[
            min(0, scale[0]),
            max(3, scale[-1]),
        ],
        x_axis_label='Year',
        y_axis_label='Avg. Bechdel Test',
    )
    
    # create a plot with the original scale [0, 1, 2, 3]
    data_source = ColumnDataSource(prepare_data_with_scale([0, 1, 2, 3]))
    fig.line(
        x='Year',
        y='rating',
        legend='Original Scale',
        source=data_source,
    )
    
    # create a plot with the new scale 
    data_source = ColumnDataSource(prepare_data_with_scale(scale))
    fig.line(
        x='Year',
        y='rating',
        # use red to make it stand out
        color='#C40000',
        legend='Modified Scale',
        source=data_source,
    )
    
    fig.legend.location = 'bottom_right'
    
    return fig

show(plot_1_alternates([0, 0, 0, 3]))

The trend over time now looks much more variable, with less convergence than before. Conversely, we can look at what happens if we consider having two women "good enough", with a scale of 0, 2.5, 2.75, 3:

In [ ]:
show(plot_1_alternates([0, 2,5, 2.75, 3]))

Now it looks like films during 1940-1980 represented a period of equal gender representation, while we have regressed from that representation in the last 30 years.

Which is the whole point: this plot is implicitly deceiving you by hiding how it converts the criteria of the Bechdel test to a number. Brain builds on this shaky foundation by attempting to find linear and quadratic tredlines on the original scale:

<table>
    <tr>
        <td>
            ![Original Average Bechdel Test over Time Plot](original-plots/1b.png)
        </td>
        <td>
            ![Original Average Bechdel Test over Time Plot](original-plots/1c.png)
        </td>
    </tr>
</table>

Although I do mildly begrudge Brian for not catching this issue in his document, this in fact *strengthens* his overall argument for open data journalism. If the source code was not available, we would not know how the results of the Bechdel test were converted to numbers, and we would not be able to identify the sleight of hand that has taken place. It's only because Brian took the effort to replicate FiveThirtyEight's analysis that we can even point to this problem.

### Improving Plot 1: Rolling Averages

We will discuss how to better represent the Bechdel test trends when we look at plot 2, but if we accept the 0, 1, 2, 3 scale for now, how might we make the data visualization more useful and/or readable?

Several issues stand out from the current plot. First, the data is noisy - there may be a multi-decade trend towards more equal gender representation, but the test results fluctuate year to year in an unpredictable manner. The most common way of getting around this problem is to use a *rolling average*. Instead of using the average test score for each year, we instead plot the average of the past $n$ years. This will prevent huge changes from year to year and help smooth out the data.

Lucky for us, pandas provides a [`rolling()` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.rolling.html) on DataFrames that does the hard work for us. Below, we plot a rolling average on top of the original noisy data:

In [ ]:
def plot_1_rolling(n):
    
    fig = figure(
        width=600, height=400,
        x_range=[1910, 2020],
        y_range=[0, 3],
        x_axis_label='Year',
        y_axis_label='Avg. Bechdel Test',
    )
    
    plot_df = raw_df.copy()
    plot_df = plot_df[['Year', 'rating']]
    plot_df = plot_df.groupby(by='Year').mean()
    plot_df = plot_df.reset_index()
    
    # plot the non-rolling data
    fig.line(
        x='Year',
        y='rating',
        legend='Yearly Average',
        source=ColumnDataSource(plot_df),
    )
    
    # calculate the rolling average
    plot_df = plot_df.rolling(n).mean()
    
    # plot the rolling data
    fig.line(
        x='Year',
        y='rating',
        legend='{} Year Rolling Average'.format(n),
        source=ColumnDataSource(plot_df),
    )
    
    fig.legend.location = 'bottom_right'
    
    return fig

show(plot_1_rolling(10))

Without any distinguishing visual characteristics, it's really hard to tell the two apart. Since the individualy yearly data is less important than the aggregate trend, we can make the former ligher while making the latter darker and thicker:

In [ ]:
def plot_1_rolling(n):
    
    fig = figure(
        width=600, height=400,
        x_range=[1910, 2020],
        y_range=[0, 3],
        title='Rolling average with n = {} years'.format(n),
        x_axis_label='Year',
        y_axis_label='Avg. Bechdel Test',
    )
    
    plot_df = raw_df.copy()
    plot_df = plot_df[['Year', 'rating']]
    plot_df = plot_df.groupby(by='Year').mean()
    plot_df = plot_df.reset_index()
    
    fig.line(
        x='Year',
        y='rating',
        # use a ligher shade of blue
        color=palettes['Blues'][5][2],
        legend='Yearly Average',
        source=ColumnDataSource(plot_df),
    )
    
    plot_df = plot_df.rolling(n).mean()
    
    fig.line(
        x='Year',
        y='rating',
        # use a darker shade of blue
        color=palettes['Blues'][5][0],
        # and a thicker line
        line_width=2,
        legend='{} Year Rolling Average'.format(n),
        source=ColumnDataSource(plot_df),
    )
    
    fig.legend.location = 'bottom_right'
    
    return fig

show(plot_1_rolling(10))

Technical note: We are actually using an average of averages - that is, we are averaging the average test scores each year. This is *different* from directly averaging the test scores for all films in 10 years. This difference will be particularly noticeable if one of the ten years has a lot more films than the others. In general, [the average-of-averages approach should be avoided](https://math.stackexchange.com/a/95911), but since the original averages are meaningless anyway (due to the ordinal data issue), we will roll with it for now. The next improvement we make will require us to change our approach.

This is quite a bit better. Using a rolling window of 10 years, we see that there is a dip in Bechdel test scores during the 1970s, a fact that was obscured by the noise in the data before. I am neither a film critic nor a social historian, so I won't offer guesses as to why this dip occurred.

Although rolling averages help, we do have to be careful to pick an appropriate $n$, however, that is neither too small (or the plot will remain noisy) nor too big (or the plot will hide important trends). Compare the results if we set the rolling window to 5 or 30 years:

In [ ]:
def create_plot_row(*figs):
    if len(figs) == 1:
        width = 600
    else:
        width = int(900 / len(figs))
    height = int(width * 2 / 3)
    row = []
    for fig in figs:
        fig.plot_width = width
        fig.plot_height = height
        row.append(fig)
    return gridplot([row])

show(create_plot_row(plot_1_rolling(5), plot_1_rolling(30)))

As we can see, a rolling average over 5 years is still relatively noisy, while a rolling average over 30 years almost erases any trends entirely.

For those of you looking at the code, you might have noticed that I used a `create_plot_row()` function. That function resizes any figures passed in as arguments, then returns a [`gridplot()`](http://bokeh.pydata.org/en/latest/docs/reference/layouts.html#bokeh.layouts.gridplot) to show them side by side.

## Improving Plot 1: Showing Variance

The new plot with a rolling average of 10 years is a definite improvement over the original, but we can include more information into the visualization. [Edward Tufte](https://en.wikipedia.org/wiki/Edward_Tufte), the great information designer, is fond of making sure that a chart is *data dense* - that is, it uses the available space to show as much data as can be meaningfully understood. Much of our plot, for example, is taken up by white space; is there a better use of that area?

One good use of that space is to indicate something about the variation in values that make up the average. A common way to do this is to show the [quartiles](https://en.wikipedia.org/wiki/Quartile) - the 25%, 50%, and 75% values - with a [box plot](https://en.wikipedia.org/wiki/Box_plot). In our case, however, these values could only be 0, 1, 2, or 3, so they are not particularly useful.

The other obvious piece of data to include is the standard deviation of Bechdel test scores for a given decade. In short, the standard deviation measures how clustered a set of values are; the more clustered they are, the lower the standard deviation. This takes into account both the number of films (which Brian originally represented using line width) as well as their Bechdel scores.

Since we are now calculating the standard deviation of Bechdel test scores across films, the average-of-averages approach from before is no longer valid. Instead, we will have to manually group the films by rolling decade to calculate the true average and the standard deviation:

In [ ]:
def plot_1_stdev(n, num_stdev=0):
    
    fig = figure(
        width=600, height=400,
        x_range=[1910, 2020],
        y_range=[0, 3],
        x_axis_label='Year',
        y_axis_label='Avg. Bechdel Test',
    )
    
    plot_df = raw_df.copy()
    plot_df = plot_df[['Year', 'rating']]
    
    # save the original noisy averages
    noisy_df = plot_df.groupby(by='Year').mean()
    noisy_df = noisy_df.reset_index()

    # manually calculate the mean and stdev of each decade
    # for each unique year, find all films within +/- (n/2) years
    # store the statistics for that year then convert back to a DataFrame
    rolling = []
    for year in plot_df['Year'].unique():
        decade_df = plot_df[(year - (n/2) <= plot_df['Year']) & (plot_df['Year'] <= year + (n/2))]
        stats = decade_df['rating'].describe()
        rolling.append([year, stats['mean'], stats['std']])
    plot_df = pd.DataFrame(rolling, columns=['Year', 'mean', 'stdev'])
    plot_df = plot_df.dropna().sort_values('Year')
    
    # calculate the lower and upper error bars
    plot_df['bottom'] = plot_df['mean'] - num_stdev * plot_df['stdev']
    plot_df['top'] = plot_df['mean'] + num_stdev * plot_df['stdev']
    
    data_source = ColumnDataSource(plot_df)
    
    # add the standard deviation first, since they will smear out
    if num_stdev > 0:
        fig.vbar(
            x='Year',
            bottom='bottom',
            top='top',
            width=1,
            color=palettes['Blues'][5][4],
            legend='+/- {:.2f} Std.'.format(num_stdev),
            source=data_source,
        )
    
    # add the original noisy data second
    fig.line(
        x='Year',
        y='rating',
        color=palettes['Blues'][5][2],
        legend='Yearly Average',
        source=ColumnDataSource(noisy_df),
    )
    
    # add the rolling average last, so it shows up on top
    fig.line(
        x='Year',
        y='mean',
        color=palettes['Blues'][5][0],
        line_width=2,
        legend='{} Year Rolling Average'.format(n),
        source=data_source,
    )
    
    fig.legend.location = 'bottom_right'
    
    return fig

show(plot_1_stdev(10))

It is subtle, but this plot is different from the previous rolling average plot, most noticeably in the dip around 1970 (the previous plot didn't have a peak in the middle of the valley). Still, the overall up and down trend seems to hold

The standard way of depicting standard deviation is with error bars, although the opposite is not true (that is, [not all error bars depict standard deviation](https://en.wikipedia.org/wiki/Error_bar)). Because our plot shows data over time, however, we can instead indicate the region within some number of standard deviations. For example, the plot below shows the mean +/- 0.5 standard deviations:

In [ ]:
show(plot_1_stdev(10, num_stdev=0.5))

As it turns out, the standard deviation is not particularly meaningful here, as it remains around 1 throughout the dataset. This is not too surprising, given that the original numbers can only be 0, 1, 2, or 3. Later in Plot 3 when we look at continuous data, standard deviation will be more useful.

Still, this plot is again an improvement over the original recreation. The macro-level trends are much easier to identify, without the need to fit a linear or quadratic regression. At the same time, the yearly specifics are still available for the interested reader; and although the standard deviation does not change, it does remind the reader that these are summary statistics of a collection of Bechdel test results. For comparison, here is the original plot with our improved version:

In [ ]:
show(create_plot_row(
    plot_1(),
    plot_1_stdev(10, num_stdev=0.5),
))

### Making Plot 1 Interactive

## Plot 2: Bechdel Score Distribution over Time

![](original-plots/2.png)

## Plot 3: Film Budget by Bechdel Score

![](original-plots/3.png)

## Plot 4: Other Metrics by Bechdel Score

<table>
    <tr>
        <td>
            ![Original Average Bechdel Test over Time Plot](original-plots/4a.png)
        </td>
        <td>
            ![Original Average Bechdel Test over Time Plot](original-plots/4b.png)
        </td>
    </tr>
</table>

<table>
    <tr>
        <td>
            ![Original Average Bechdel Test over Time Plot](original-plots/4c.png)
        </td>
        <td>
            ![Original Average Bechdel Test over Time Plot](original-plots/4d.png)
        </td>
    </tr>
</table>